In [1]:
import numpy as np
import pandas as pd

offsets = np.mean(pd.read_csv('gyro_offset.csv').to_numpy(), axis=0)
print(offsets)

angular_rates_x = pd.read_csv('gyro_x.csv').to_numpy() - offsets[0]
angular_rates_y = pd.read_csv('gyro_y.csv').to_numpy() - offsets[1]
angular_rates_z = pd.read_csv('gyro_z.csv').to_numpy() - offsets[2]

[0.01874973 0.07373471 0.06578995]


Offsets are really far from gt offsets

In [2]:
sample_rate = 1000
angles_x = np.sum(angular_rates_x, axis=0) / sample_rate
angles_y = np.sum(angular_rates_y, axis=0) / sample_rate
angles_z = np.sum(angular_rates_z, axis=0) / sample_rate
print(f"x: {angles_x * 180 / np.pi}")
print(f"y: {angles_y * 180 / np.pi}")
print(f"z: {angles_z * 180 / np.pi}")

x: [89.258718    9.1704333  14.95849953]
y: [-16.97847894  78.58034816  -9.58001803]
z: [-17.09447971  -9.22413469  85.12186937]


In [3]:
true_angles_x = np.array([np.pi / 2, 0, 0])
true_angles_y = np.array([0, np.pi / 2, 0])
true_angles_z = np.array([0, 0, np.pi / 2])

In [4]:
Yg = np.array([angles_x, angles_y, angles_z])
Yg * 180 / np.pi

array([[ 89.258718  ,   9.1704333 ,  14.95849953],
       [-16.97847894,  78.58034816,  -9.58001803],
       [-17.09447971,  -9.22413469,  85.12186937]])

In [5]:
Ug = np.array([true_angles_x, true_angles_y, true_angles_z])

In [6]:
invUg = np.linalg.inv(Ug)
ST = np.linalg.cholesky((Yg @ invUg) @ (Yg @ invUg).T).T
ST

array([[ 1.01074305, -0.11459167, -0.04117802],
       [ 0.        ,  0.89225429, -0.17825395],
       [ 0.        ,  0.        ,  0.95270445]])

In [7]:
import scipy

S, T = scipy.linalg.lu(ST, permute_l=True)

In [8]:
print(f"Scale factors matrix: {S}")
print(f"Orthogonality matrix: {T}")

Scale factors matrix: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Orthogonality matrix: [[ 1.01074305 -0.11459167 -0.04117802]
 [ 0.          0.89225429 -0.17825395]
 [ 0.          0.          0.95270445]]


In [9]:
from numpy.linalg import inv

M = inv(T) @ inv(S) @ Yg @ inv(Ug)

In [10]:
print(f"Misalignment matrix: {M}")

Misalignment matrix: [[ 0.94461362  0.20493305  0.21384422]
 [-0.25126007  0.95705756  0.07903262]
 [-0.19936788 -0.10757836  0.99275127]]
